In [7]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using Plots
using StatsPlots
using Dates
using HDF5

In [ ]:
Hᴸᴹᴱ = [ 25.126        0
0             0    ]
γᶠ = parse(Float64, γᵢ)

Aᴸᴹᴱ = [ 0    √(γᶠ)
0     0. + 0im  ]

tᵉᶠ = convert.(Float64, tᵉ); tᵍᶠ = convert.(Float64, tᵍ); 
tˣᶠ = convert.(Float64, tˣ); tʸᶠ = convert.(Float64, tʸ); 

ρᵉᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵉ[1], tᵉᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
ρᵍᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρᵍ[1], tᵍᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
ρˣᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρˣ[1], tˣᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
ρʸᴸᴹᴱ = LiPoSID.Lindblad_time_evolution(basis, ρʸ[1], tʸᶠ, Hᴸᴹᴱ, [Aᴸᴹᴱ])

In [8]:
function DMD_SVD(Y, r, Δt)
    
        X₋ = Y[:,1:end-1]
        X₊ = Y[:,2:end]
        U, Σ, V = svd(X₋)
        
        Uʳ = U[:, 1:r] #12 x 4
        Σʳ = diagm(Σ[1:r])
        Vʳ = V[:, 1:r]
        Ã = Uʳ' * X₊ * Vʳ / Σʳ
        Λ, W = eigen(Ã)
        Φ = X₊ * Vʳ / Σʳ * W
        Ω = log.(Λ)/Δt
        x₁ = X₋[:,1]
        b₁ = Φ \ x₁
        
        return Φ, Ω, b₁
    
end    

function DMD_reconstruct(Φ, Ω, b₁, Δt, steps)
    
        Yᵈᵐᵈ = hcat([real.(Φ * (b₁ .* exp.(Ω * (i * Δt)))) for i in 0:steps]...)
        
        return Yᵈᵐᵈ
    
end


function DMD_basis_SpinBoson(γᵢ, n)
    data_dir = "DATA/"

    γᵢ = string(γᵢ)

    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
    
    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    bᵉ = LiPoSID.bloch(ρᵉ[1:lᵐᵃˣ])
    bᵍ = LiPoSID.bloch(ρᵍ[1:lᵐᵃˣ])
    bˣ = LiPoSID.bloch(ρˣ[1:lᵐᵃˣ])
    bʸ = LiPoSID.bloch(ρʸ[1:lᵐᵃˣ])

    Y = [bᵉ; bᵍ; bˣ; bʸ]

    t = convert.(Float64, tᵉᶠ[1:lᵐᵃˣ])
    Δt = t[2]-t[1]
    
    Φ, Ω, b₁ = DMD_SVD(Y, n, Δt)
    Yᴰᴹᴰ = DMD_reconstruct(Φ, Ω, b₁, Δt, length(t))

    bᵉᴰᴹᴰ = Yᴰᴹᴰ[1:3,:]
    bᵍᴰᴹᴰ = Yᴰᴹᴰ[4:6,:]
    bˣᴰᴹᴰ = Yᴰᴹᴰ[7:9,:]
    bʸᴰᴹᴰ = Yᴰᴹᴰ[10:12,:]

    ρᵉᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bᵉᴰᴹᴰ)
    ρᵍᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bᵍᴰᴹᴰ)
    ρˣᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bˣᴰᴹᴰ)
    ρʸᴰᴹᴰ = LiPoSID.rho_series_from_bloch(bʸᴰᴹᴰ)


    return [ρᵉᴰᴹᴰ, ρᵍᴰᴹᴰ, ρˣᴰᴹᴰ, ρʸᴰᴹᴰ], t[1:lᵐᵃˣ]
    
end

function ERA_basis_SpinBoson(γᵢ, n)

        data_dir = "DATA/"

        γᵢ = string(γᵢ)
    
        ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
        ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
        ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
        ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)
        
        lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
        lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series
    
        bᵉ = LiPoSID.bloch(ρᵉ[1:lᵐᵃˣ])
        bᵍ = LiPoSID.bloch(ρᵍ[1:lᵐᵃˣ])
        bˣ = LiPoSID.bloch(ρˣ[1:lᵐᵃˣ])
        bʸ = LiPoSID.bloch(ρʸ[1:lᵐᵃˣ])
    
        Y = [bᵉ; bᵍ; bˣ; bʸ]

        t = convert.(Float64, tᵉᶠ[1:lᵐᵃˣ])
        Δt = t[2]-t[1]

        A, C, x₀, Σ = LiPoSID.lsid_n_ACx0Σ(Y, Δt, n) 

        Yᴱᴿᴬ = LiPoSID.propagate_LTI(A, C, x₀, n, length(t))

        bᵉᴱᴿᴬ = Yᴱᴿᴬ[1:3,:]
        bᵍᴱᴿᴬ = Yᴱᴿᴬ[4:6,:]
        bˣᴱᴿᴬ = Yᴱᴿᴬ[7:9,:]
        bʸᴱᴿᴬ = Yᴱᴿᴬ[10:12,:]

        ρᵉᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bᵉᴱᴿᴬ)
        ρᵍᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bᵍᴱᴿᴬ)
        ρˣᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bˣᴱᴿᴬ)
        ρʸᴱᴿᴬ = LiPoSID.rho_series_from_bloch(bʸᴱᴿᴬ)

        return [Fᵉ, Fᵍ, Fˣ, Fʸ], t[1:lᵐᵃˣ], A
        
end

ERA_fidelity_4x4 (generic function with 1 method)

In [9]:
parentdir = pwd()
tests_dir = parentdir*"/TESTS/"

"/home/zah/github/POP23/TESTS/"

In [10]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
#models_file = "DMDvsERA_LME_trn4_models_"*date_and_time_string * ".h5"
tests_data_file_name = "DMDvsERA_LME_trn4_tst4_"*date_and_time_string * ".h5"

γ = ["0.0", "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

println("Coupling levels to be avaluated γ ∈ ", γ)

for n in 3:5

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        rank_group = create_group(fid, "rank_"*string(n)) # create rank group
    end 

    for γᵢ in  γ

        ρᴰᴹᴰ, tᴰᴹᴰ,  Λᴰᴹᴰ = DMD_basis_SpinBoson(γᵢ, n)
        ρᴱᴿᴬ, tᴱᴿᴬ,  Λᴱᴿᴬ = ERA_basis_SpinBoson(γᵢ, n)

        ρᵉᴰᴹᴰ, ρᵍᴰᴹᴰ, ρˣᴰᴹᴰ, ρʸᴰᴹᴰ = ρᴰᴹᴰ
        ρᵉᴱᴿᴬ, ρᵍᴱᴿᴬ, ρˣᴱᴿᴬ, ρʸᴱᴿᴬ = ρᴱᴿᴬ


        for df_trn in train_files # loop over initial states

            ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
            
            if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
                
            ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])

            b₀ᵗʳⁿ = LiPoSID.bloch(ρᵗʳⁿ[1])
            kᵉᴰᴹᴰ, kᵍᴰᴹᴰ, kˣᴰᴹᴰ, kʸᴰᴹᴰ = b₀ᵗʳⁿ * [bᵉᴰᴹᴰ, bᵍᴰᴹᴰ, bˣᴰᴹᴰ, bʸᴰᴹᴰ]

            
            
            Fʸ = LiPoSID.fidelity_series(basis, ρʸᴸᴹᴱ[1:lᵐᵃˣ], ρʸᴱᴿᴬ[1:lᵐᵃˣ])



        Fᴰᴹᴰ, t = DMD_fidelity_4x4(γᵢ, n)
        Fᴱᴿᴬ, t = ERA_fidelity_4x4(γᵢ, n) # Fᵉ, Fᵍ, Fˣ, Fʸ 

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            rank_group = open_group(fid, "rank_"*string(n)) # open rank group

            γ_group = create_group(rank_group, "gamma_"*γᵢ) # create coupling group

            γ_group["Fe_dmd_lme"] = convert.(Float64, Fᴰᴹᴰ[1])
            γ_group["Fg_dmd_lme"] = convert.(Float64, Fᴰᴹᴰ[2])
            γ_group["Fx_dmd_lme"] = convert.(Float64, Fᴰᴹᴰ[3])
            γ_group["Fy_dmd_lme"] = convert.(Float64, Fᴰᴹᴰ[4])

            γ_group["Fe_era_lme"] = convert.(Float64, Fᴱᴿᴬ[1])
            γ_group["Fg_era_lme"] = convert.(Float64, Fᴱᴿᴬ[2])
            γ_group["Fx_era_lme"] = convert.(Float64, Fᴱᴿᴬ[3])
            γ_group["Fy_era_lme"] = convert.(Float64, Fᴱᴿᴬ[4])   
            
            γ_group["time"] = convert.(Float64, t) 

            
        end
    end

end


Coupling levels to be avaluated γ ∈ ["0.0", "0.079477", "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]


In [11]:
tests_data_file_name

"DMDvsERA_LME_trn4_tst4_2023-Aug-19_at_09-37.h5"